# Toxic Comment Detection with PyTorch LSTM

In this kernel, I implement LSTM for toxicity prediction using only PyTorch (and torchtext) whenever possible. I have been a keras user since forever, but I notice more and more new architectures are built through PyTorch. Thus, the main purpose of building this kernel for me is to learn PyTorch. I leave some comments to explain what each section is trying to do, hopefully it can help you too.

I incorporate many techniques that I have learned from other excellent kernels, shoutout to:
- @bminixhofer: https://www.kaggle.com/bminixhofer/simple-lstm-pytorch-version
- @christofhenkel: https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part1-eda

Having been inspired by those kernels, there are some modifications:
- based on my EDA kernel (https://www.kaggle.com/yosefardhito/eda-annotations-lexical-and-bag-of-words), I ignore comments with both low toxicity annotator count and toxic target value between 0.4 and 0.6 because I noticed how comments with target between that range has a questionable label even for me, let alone the model. For auxiliary attributes, I use only 'insult' because it is the only attributes significantly correlated with the target column. 
- I add a simple train/dev/test split process so I can experiment with the training data alone.
- Paddings are added at the beginning instead of after the comment.
- Slight modification of the Simple LSTM model on the concatenation.

Why LSTM? I know from the stories of the top leaderboards that BERT and XLNet provide the best result. However, since the goal of this kernel is to learn, I prefer training a model with simpler and modifiable architecture. I want to emphasize on learning how to do it in PyTorch instead. Surely, pre-trained BERT and XLNet are next to be mastered!

In [2]:
import string
import re
import operator
import pickle
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook as tqdm; tqdm.pandas()

from nltk.tokenize.treebank import TreebankWordTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from torchtext.data import Example, Field, Dataset, LabelField, Iterator, BucketIterator
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# dataframe options to display the whole comments
pd.set_option('display.max_colwidth', -1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

device

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


'cuda'

# 1. Data Loading

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Downloading and get the files manually with out drive
# Would require kaggle.json auth file to be uploaded
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

## 1.1. Load Dataset

In [0]:
# load only necessary fields to reduce memory footprint
fields = ['id', 'comment_text', 'toxicity_annotator_count', 'insult', 'target']
# load training set
train_df = pd.read_csv("./drive/My Drive/train.csv.zip", usecols=fields).set_index('id')
X_train = train_df['comment_text']
y_train = train_df['target'] >= 0.5  # create binary target column
train_df['is_toxic'] = y_train
train_df['is_insult'] = train_df['insult'] >= 0.5
# split training set to train/dev/test set with stratified strategy
X_train, X_dev_test, y_train, y_dev_test = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=1337)
X_dev, X_test, y_dev, y_test = train_test_split(X_dev_test, y_dev_test, stratify=y_dev_test, test_size=0.5, random_state=1338)
# load submission set
test_df = pd.read_csv("./drive/My Drive/test.csv.zip").set_index('id')
X_submission = test_df['comment_text']

In [4]:
print("Length train: {:,}; dev: {:,}; test: {:,}; submission: {:,}".format(
    len(X_train), len(X_dev), len(X_test), len(X_submission)
))

Length train: 1,443,899; dev: 180,487; test: 180,488; submission: 97,320


## 1.2. Load Embedding

I only try glove embeddings during this experiment. Pickled version is used to reduce time for loading.

In [6]:
!unzip './drive/My Drive/glove.840B.300d.zip'


Archive:  ./drive/My Drive/glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [0]:
# Downloading the files manually from stanford
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip 'glove.840B.300d.zip'



In [7]:
# def load_embeddings(path):
#     with open(path,'rb') as f:
#         emb_arr = pickle.load(f)
#     return emb_arr

def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

EMBEDDING_PATH = "glove.840B.300d.txt"
embeddings = loadGloveModel(EMBEDDING_PATH)

Loading Glove Model
Done. 2196016  words loaded!


In [0]:
import pickle

# with open('filename.pickle', 'wb') as handle:
#     pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./drive/My Drive/filename.pickle', 'rb') as handle:
    embeddings = pickle.load(handle)



# 2. Data Preprocessing

## 2.1. Filtering

Based on the EDA, I choose to ignore comments with target value between 0.4 and 0.6. Also, because the training set consists of an abundant number of comments with target value = 0, I decided only to sample comments that have both really low value for target and insult.

In [0]:
# take all comments with toxicity annotator count >= 10 ** 1.5
# train_trust_df = train_df[(train_df.index.isin(X_train.index)) & (train_df['toxicity_annotator_count'] >= 10 ** 1.5)]
# filter comments with toxicity annotator count < 10 ** 1.5 based on the target value
# train_chosen_df = train_df[train_df.index.isin(X_train.index) & (train_df['toxicity_annotator_count'] < 10 ** 1.5)]
train_chosen_df = train_df[train_df.index.isin(X_train.index)]
# for safe comments, take only those with both target and insult value < 0.2 ("very safe")
train_chosen_very_safe_df = train_chosen_df[(train_chosen_df['target'] < 0.2) & (train_chosen_df['insult'] < 0.2)]
# for toxic comments, ignore those between 0.4 and 0.5
train_chosen_toxic_df = train_chosen_df[train_chosen_df['target'] >= 0.6]
train_chosen_others_df = train_chosen_df[(train_chosen_df['target'] >= 0.2) & (train_chosen_df['target'] < 0.4)]
# take maximum 1mil to fit processing within 2-hour time limit for submission and the given memory size
# n_very_safe = min(len(train_chosen_very_safe_df), 
#                   1100032 - len(train_trust_df) - len(train_chosen_others_df) - len(train_chosen_toxic_df))
n_very_safe = min(len(train_chosen_very_safe_df), 
                  1100032 - len(train_chosen_others_df) - len(train_chosen_toxic_df))
train_chosen_very_safe_sample_df = train_chosen_very_safe_df.sample(n=n_very_safe, random_state=1337)
# get all valid comment indices
train_indices = [
    #*train_trust_df.index.values,
    *train_chosen_very_safe_sample_df.index.values, 
    *train_chosen_others_df.index.values, 
    *train_chosen_toxic_df.index.values
]
X_train = X_train[X_train.index.isin(train_indices)]
X_dev = X_dev.sample(frac=1, random_state=1337)
X_test = X_test.sample(frac=1, random_state=1337)

In [7]:
print("Length train: {:,}; dev: {:,}; test: {:,}; submission: {:,}".format(
    len(X_train), len(X_dev), len(X_test), len(X_submission)
))

Length train: 1,100,032; dev: 180,487; test: 180,488; submission: 97,320


## 2.2. Prepare Accepted Characters

Because I use embeddings, there are some symbols that do not have the respective embedding vectors. In this section, I check all characters used in the training set. The character set is compared to glove embeddings symbols afterward. Instead of using slow `replace()` function, we will use `translate()`. Basically, `translate()` take a dictionary of character mapping as input. The key of the dictionary is the ordinal of the characters.

In [8]:
def build_character_set(sentences, verbose=True):
    characters_in_dataset = {}
    for sentence in tqdm(sentences, disable=(not verbose)):
        for character in sentence:
            try:
                characters_in_dataset[character] += 1
            except KeyError:
                characters_in_dataset[character] = 1
    return characters_in_dataset

train_charset = build_character_set(X_train)
# build all accepted characters
latin_similar = " ’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
white_list = string.ascii_letters + string.digits + latin_similar
# check all symbols that have embeddings to keep it
glove_symbols = [symbol for symbol in embeddings if len(symbol) == 1 and symbol not in white_list]
char_translate_map = {}
# isolate symbols that have embeddings vector in glove
for symbol in glove_symbols:
    char_translate_map[ord(symbol)] = ' ' + symbol + ' '
# remove all unknown symbols
unknown_symbols = [c for c in train_charset if c not in white_list and c not in glove_symbols]
for symbol in unknown_symbols:
    char_translate_map[ord(symbol)] = ''
# replace newline characters with normal separator (blank space)
char_translate_map[ord('\n')] = ' ' 

## 2.3. Text Preprocessing

In [9]:
def find_in_embedding(word, embedding):
    '''
    find the word's vector in the embedding. 
    If there is no exact word found, try also with lower case and title case.
    In case there is no entry found in the embedding, replace with the unknown token (<unk>)
    '''
    if word in embedding:
        return word, embedding[word]
    elif word.lower() in embedding:
        return word.lower(), embedding[word.lower()]
    elif word.title() in embedding:
        return word.title(), embedding[word.title()]
    else:
        return ',', embedding[',']

def preprocess(comment, translate_map, tokenizer, emb):
    # translate symbols
    preprocessed_comment = comment.translate(translate_map)
    # handle quote at the beginning
    # because we accepted quotes as characters, sometimes we encounter token such as "'we".
    # with this step, we transform "'we" into "' we" and such that it can be tokenized as 2 tokens.
    preprocessed_comment = [("' " + token[1:]) if len(token) > 0 and token[0] == "'" else token for token in preprocessed_comment.split(' ')]
    # tokenize with Treebank tokenizer
    tokenized_comment = tokenizer.tokenize(' '.join(preprocessed_comment))
    # check and find each token in embeddings
    tokenized_comment = [find_in_embedding(token, emb)[0] for token in tokenized_comment]
    return ' '.join(tokenized_comment)

tokenizer = TreebankWordTokenizer()
X_train = X_train.progress_apply(lambda comment: preprocess(comment, char_translate_map, tokenizer, embeddings))
X_dev = X_dev.progress_apply(lambda comment: preprocess(comment, char_translate_map, tokenizer, embeddings))
X_test = X_test.progress_apply(lambda comment: preprocess(comment, char_translate_map, tokenizer, embeddings))
X_submission = X_submission.progress_apply(lambda comment: preprocess(comment, char_translate_map, tokenizer, embeddings))

In [10]:
X_train.head()

id
6183788    So why is the clinton / fbi / russian colusion story nowhere on ADN ?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

# 3. Data Preparation for PyTorch

## 3.1. Create PyTorch Datasets

Before we can train a pytorch model, we have to transform the input to the form that is expected by the model. One option is to use keras text preprocessing, but I'd rather stick with pytorch family, i.e. torchtext. First, we want to transform the input into a torchtext `Dataset`. A `Dataset` is a collection of `Example`, which consists of `Field`. More info: https://torchtext.readthedocs.io/en/latest/data.html

A `Field` has many useful options for NLP, among many of them are:
- `pad_first=True` to set padding tokens added at the beginning instead of after the comment
- `fix_length=None`, a default, to have varying length
- `sequential=True` means the field is to be treated a sequential data. For text, set it to `True`, set otherwise for labels.
- `use_vocab=True` whether to build vocabulary based on this field. Set to `True` for the text fields.

The downside of this approach is the slowness due to the iteration to create the `Example` and `Dataset`. There should be a better way to convert the input to torchtext `Dataset`. There is also a `TabularDataset` class which can transform the whole dataset at one, but I am not aware how to include the preprocessing required without exporting the preprocessed dataset beforehand. 

In [11]:
def prepare_torch_dataset(comments, target, fields):
    '''
    transform a series `comments` to torchtext `Dataset`
    '''
    df = pd.DataFrame(comments).join(target[['is_toxic', 'is_insult']], how='left')
    train_examples = [Example.fromlist(i, fields) for i in tqdm(df.values.tolist())]
    return Dataset(train_examples, fields)

# define the Fields
ID = Field(sequential=False, use_vocab=False, dtype=torch.long)
TEXT = Field(sequential=True, tokenize='spacy', pad_first=True)
LABEL = LabelField(sequential=False, use_vocab=False, dtype=torch.float32)
fields = [("comment_text", TEXT), ("target", LABEL), ("insult", LABEL)]
# prepare train/dev/test datasets as torch `Dataset`
train_dataset = prepare_torch_dataset(X_train, train_df, fields)
dev_dataset = prepare_torch_dataset(X_dev, train_df, fields)
test_dataset = prepare_torch_dataset(X_test, train_df, fields)
# prepare submission dataset as torch `Dataset`
test_df['comment_text'] = X_submission
subm_fields = [("id", ID), ("comment_text", TEXT)]  # notice that there is no label field for the submission dataset
subm_examples = [Example.fromlist(i, subm_fields) for i in tqdm(test_df.reset_index().values.tolist())]
subm_dataset = Dataset(subm_examples, subm_fields)

Quite handily, torchtext also provides us with `build_vocab()` capability. This function helps us with the mapping between token and integer, useful for embeddings. Additionally, it contains the frequency of each word as well. `max_size` parameter determines maximum number of words registered in the vocabulary, truncated based on the frequency (top `max_size` words with the highest frequency will be used).

In [0]:
def build_matrix(vocab_itos, emb):
    embedding_matrix = np.zeros((len(vocab_itos), 300))
    for i, word in enumerate(vocab_itos):
        embedding_matrix[i] = find_in_embedding(word, emb)[1]
    return embedding_matrix

TEXT.build_vocab(train_dataset, dev_dataset, max_size=60000)
embedding_matrix = build_matrix(TEXT.vocab.itos, embeddings)

In [16]:
for i in range(20):
    word = TEXT.vocab.itos[i]
    print("id={}, token='{}', freq={}".format(i, word, TEXT.vocab.freqs[word]))

id=0, token='<unk>', freq=0
id=1, token='<pad>', freq=0
id=2, token='.', freq=4914165
id=3, token='the', freq=3043906
id=4, token=',', freq=2730407
id=5, token='to', freq=1868630
id=6, token='`', freq=1698286
id=7, token='and', freq=1518186
id=8, token='of', freq=1441766
id=9, token='a', freq=1341675
id=10, token='is', freq=1087605
id=11, token='in', freq=942567
id=12, token='that', freq=903165
id=13, token='-', freq=806483
id=14, token='I', freq=742824
id=15, token='for', freq=659848
id=16, token='it', freq=610435
id=17, token='you', freq=606008
id=18, token='?', freq=554502
id=19, token='are', freq=532599


## 3.2. Sequence Bucketing

In this dataset, the comment length ranges from 1 word to hundreds. If we take the maximum length of a comment and pad everything according to the maximum length, it won't be effective for training. Sequence bucketing can help reduce the time required to train. We reorder the comments based on their length, such that comments with similar length is more likely to belong on the same batch. At each batch, pad every comments to the maximum comment length on that batch. Therefore, we do not need to pad 1-word comment to the max length, and training can be much quicker.

I avoid truncating any of the comment because it is misleading to decide either truncate from beginning or from end. A comment can be toxic just because 1 or 2 words it contains and those words can be at the beginning or at the end. Consequently, some outlier lengthy comments will affect the padding of much shorter ones.

In torchtext, we can do sequence bucketing with `BucketIterator` class. The important parameters are `sort_key` and `sort_within_batch`. Basically, `sort_key=True` does what we expect from sequence bucketing and it is no longer necessary to also set `sort_within_batch=True` because the comments in one batch will be padded to the maximum length of that batch anyway.

In [17]:
train_iter, dev_iter, test_iter, subm_iter = BucketIterator.splits(
    (train_dataset, dev_dataset, test_dataset, subm_dataset),
    batch_sizes=(64, 64, 64, 64),
    device=0,
    sort_key=lambda comment: len(comment.comment_text),
    sort_within_batch=False,
    repeat=False  # do not repeat any inputs
)

class BatchWrapper:
    '''
    helper wrapper to iterate over `BucketIterator` during training and return (x, y).
    from: https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
    '''
    
    def __init__(self, dl, x_var, y_vars):
        self.dl, self.x_var, self.y_vars = dl, x_var, y_vars

    def __iter__(self):
        for batch in self.dl:
            # return x_var attribute of the data loader (dl) as x
            x = getattr(batch, self.x_var)
            # return label attributes of the data loader (dl) as y
            y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y_vars], dim=1)
            yield (x, y)

    def __len__(self):
        return len(self.dl)

train_dl = BatchWrapper(train_iter, "comment_text", ["target", "insult"])
dev_dl = BatchWrapper(dev_iter, "comment_text", ["target", "insult"])
test_dl = BatchWrapper(test_iter, "comment_text", ["target", "insult"])

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


# 4. Model Training

## 4.1. Network Architecture

The architecture of the network follows Simple LSTM architecture (https://www.kaggle.com/thousandvoices/simple-lstm) with slight modification. These are the layers:
1. Embedding layer (1x60000) &rarr; (60000x300)
2. Spatial dropout of embedding with dropout rate 0.3
3. BiLSTM 1: 300 &rarr; 150 * 2 (bi-directional)
4. BiLSTM 2: 300 &rarr; 300
5. Concatenation of last state of BiLSTM (both direction) with average and max pooling of all hidden state: 300 + 300 + 300 = 900
6. Fully-connected 1: 900 of concat result &rarr; 900
7. Fully-connected 2: 900 of concat result &rarr; 900
8. Add result of layer 5, 6, and 7: 900
9. Dropout rate of 0.1 from layer 8
10. Output layer for target: 768 &rarr; 1
11. Output layer for insult: 768 &rarr; 1

In [18]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

class LSTM(nn.Module):
    def __init__(self, emb_matrix, lstm_units=150):
        super().__init__()
        # LAYER 1: EMBEDDING, non-trainable
        self.embedding = nn.Embedding(*emb_matrix.shape)
        # use glove embeddings
        self.embedding.weight = nn.Parameter(torch.tensor(emb_matrix, dtype=torch.float32), requires_grad=False)
        # LAYER 2: SPATIAL DROPOUT
        self.embedding_dropout = SpatialDropout(0.3)
        # LAYER 3: LSTM, bi-directional, output = 2 * lstm_units
        self.lstm_units = lstm_units
        self.lstm1 = nn.LSTM(embedding_matrix.shape[1], lstm_units, bidirectional=True)
        # LAYER 4: LSTM, bi-directional, output = 2 * lstm_units
        self.lstm2 = nn.LSTM(2 * lstm_units, lstm_units, bidirectional=True)
        # LAYER 5: CONCAT, no object created
        # LAYER 6: FC1
        self.linear1 = nn.Linear(3 * 2 * lstm_units, 3 * 2 * lstm_units)
        # LAYER 7: FC2
        self.linear2 = nn.Linear(3 * 2 * lstm_units, 3 * 2 * lstm_units)
        # LAYER 8: ADDITION of layer outputs, no object created
        # LAYER 9: DROPOUT
        self.dropout = nn.Dropout(0.1)
        # LAYER 10: OUTPUT LAYER for target
        self.target_out = nn.Linear(3 * 2 * lstm_units, 1)
        # LAYER 11: OUTPUT LAYER for insult
        self.aux_out = nn.Linear(3 * 2 * lstm_units, 1)

    def forward(self, seq):
        # get embedding vector of every word
        h_emb = self.embedding_dropout(self.embedding(seq))
        # walk through the first bi-directional LSTM
        h_lstm1, _ = self.lstm1(h_emb)
        # use output of previous LSTM as input for the second bi-directional LSTM
        h_lstm2, _ = self.lstm2(h_lstm1)
        # get the latest state of LSTM 2
        # because it is bi-directional, get the first half output of the last state (LSTM direction forward)
        # and the second half output of the first state (LSTM direction backward)
        h_lstm_last = h_lstm2[-1, :, :self.lstm_units]
        h_lstm_first = h_lstm2[0, :, self.lstm_units:]
        # concat both half
        h_bi_lstm = torch.cat((h_lstm_first, h_lstm_last), -1)
        # get average and max pool of all hidden state from LSTM 2
        avg_pool = torch.mean(h_lstm2, 0)
        max_pool, _ = torch.max(h_lstm2, 0)
        # concat last state with pooling
        h_conc = torch.cat((h_bi_lstm, max_pool, avg_pool), 1)
        # fully-connected layer with last state and the pooling
        h_lin1 = F.relu(self.linear1(h_conc))
        h_lin2 = F.relu(self.linear2(h_conc))
        # add concat with output of fully-connected layer
        h_conc_add = h_conc + h_lin1 + h_lin2
        # another dropout to force model check last state, average pool, and max pool values
        h_conc_do = self.dropout(h_conc_add)
        # get linear prediction of target and insult
        # we do not use sigmoid here because pyTorch provides a better loss calculation
        # by combining sigmoid with cross-entropy
        target_out = self.target_out(h_conc_do)
        aux_out = self.aux_out(h_conc_do)
        out = torch.cat([target_out, aux_out], 1)
        return out
    
model = LSTM(embedding_matrix).to(device)
model

LSTM(
  (embedding): Embedding(60002, 300)
  (embedding_dropout): SpatialDropout(p=0.3, inplace=False)
  (lstm1): LSTM(300, 150, bidirectional=True)
  (lstm2): LSTM(300, 150, bidirectional=True)
  (linear1): Linear(in_features=900, out_features=900, bias=True)
  (linear2): Linear(in_features=900, out_features=900, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (target_out): Linear(in_features=900, out_features=1, bias=True)
  (aux_out): Linear(in_features=900, out_features=1, bias=True)
)

## 4.2. Training

In [20]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def evaluate(model, dl, loss_func):
    '''
    evaluation steps for dev and test set
    '''
    running_loss = 0.0
    model.eval()  # turn on evaluation mode
    all_preds = {'y_pred' : [], 'y_true' : []}
    for x, y in dl: # iterate over mini-batches
        x, y = x.to(device), y.to(device)  # transfer x and y to GPU when it is available
        preds = model(x)  # get the prediction from the model
        loss = loss_func(preds, y)  # calculate loss values
        # add all predictions and y_true to a dictionary for calculating the evaluation metrics
        all_preds['y_pred'] = [*all_preds['y_pred'], *sigmoid(preds.detach().cpu().numpy()[:,0].ravel())]
        all_preds['y_true'] = [*all_preds['y_true'], *y.detach().cpu().numpy()[:,0].ravel()]
        # add current mini-batch loss to the total loss
        running_loss += loss.item() / len(dl)
    # create a binary prediction to calculate accuracy
    all_preds['y_pred_bin'] = [1 if pred >= 0.5 else 0 for pred in all_preds['y_pred']]
    print('\tEval Loss: {:.4f}, Eval AUC: {:.4f}, Eval Accuracy: {:.4f}'.format(
        loss,
        roc_auc_score(all_preds['y_true'], all_preds['y_pred']),
        accuracy_score(all_preds['y_true'], all_preds['y_pred_bin'])
    ))
    # confusion analysis, especially helpful for imbalanced dataset
    tn, fp, fn, tp = confusion_matrix(all_preds['y_true'], all_preds['y_pred_bin']).ravel()
    print('\tEval CM: tn={:,}({:.2f}%), tp={:,}({:.2f}%), fn={:,}({:.2f}%), fp={:,}({:.2f}%)'.format(
        tn, tn * 100 / len(all_preds['y_pred_bin']),
        tp, tp * 100 / len(all_preds['y_pred_bin']),
        fn, fn * 100 / len(all_preds['y_pred_bin']),
        fp, fp * 100 / len(all_preds['y_pred_bin'])
    ))
    
    
def train(model, train, dev, n_epochs, loss_func, lr):
    '''
    model training steps for the training set. Output dev metrics at each epoch.
    PyTorch requires implementation of training procedure.
    '''
    opt = optim.Adam(model.parameters(), lr=lr)  # optimizer
    # degrade learning rate at every epoch with lr scheduler
    scheduler = optim.lr_scheduler.StepLR(opt, step_size=1, gamma=0.3)
    for epoch in range(1, n_epochs + 1):
        print('Epoch: {}'.format(epoch))
        model.train()  # during training, dropout is activated
        running_loss = 0.0
        for x, y in tqdm(train):
            opt.zero_grad()  # necessary at the beginning of each mini-batch for pytorch
            x, y = x.to(device), y.to(device)  # transfer to GPU when available
            preds = model(x)  # get prediction
            loss = loss_func(preds, y)  # calculate loss
            loss.backward()  # backpropagation
            opt.step()  # calculate optimizer with adam
            running_loss += loss.item() / len(train_dl)  # add current mini-batch loss to total loss
        print('\tTraining Loss: {:.4f}'.format(running_loss))
        evaluate(model, dev, loss_func)  # check dev scores
        scheduler.step()  # reduce learning rate

# Binary Cross-Entropy With Logits Loss: optimized loss function for binary classification
# when using this loss function, do not use sigmoid activation function in the output layer of the model.
bce_loss = nn.BCEWithLogitsLoss(
    pos_weight=torch.FloatTensor([1]).cuda(),   # I tried to penalize mis-classified positive instance more
                                                # but it did not give better LB score
    reduction='mean'  # because we are predicting 2 outputs, take the mean as loss value
)
train(model, train_dl, dev_dl, 2, bce_loss, lr=8e-4)  # run training

Epoch: 1



	Training Loss: 0.0895
	Eval Loss: 0.6185, Eval AUC: 0.9595, Eval Accuracy: 0.9500
	Eval CM: tn=164,769(91.29%), tp=6,696(3.71%), fn=7,737(4.29%), fp=1,285(0.71%)
Epoch: 2



	Training Loss: 0.0730
	Eval Loss: 0.2686, Eval AUC: 0.9619, Eval Accuracy: 0.9498
	Eval CM: tn=165,002(91.42%), tp=6,420(3.56%), fn=8,013(4.44%), fp=1,052(0.58%)


In [21]:
evaluate(model, test_dl, bce_loss)  # evaluate on test set

	Eval Loss: 0.0292, Eval AUC: 0.9601, Eval Accuracy: 0.9497
	Eval CM: tn=165,048(91.45%), tp=6,370(3.53%), fn=8,064(4.47%), fp=1,006(0.56%)


In [22]:
# fine-tuning embedding layer
model.embedding.weight.requires_grad = True
train(model, train_dl, dev_dl, 1, bce_loss, lr=8e-5)

Epoch: 1



	Training Loss: 0.0678
	Eval Loss: 0.3312, Eval AUC: 0.9631, Eval Accuracy: 0.9527
	Eval CM: tn=164,173(90.96%), tp=7,781(4.31%), fn=6,652(3.69%), fp=1,881(1.04%)


In [23]:
evaluate(model, test_dl, bce_loss)  # evaluate on test set

	Eval Loss: 0.0215, Eval AUC: 0.9612, Eval Accuracy: 0.9522
	Eval CM: tn=164,168(90.96%), tp=7,695(4.26%), fn=6,739(3.73%), fp=1,886(1.04%)


The evaluation metrics used in this loop is basic AUC ROC, whereas the competition impose sub-groups AUC score in the calculation.

# 5. Submission

In [24]:
model.eval()  # turn on evaluation mode to ignore dropouts
subm_id = []
subm_preds = []
for example in tqdm(subm_iter):
    x_id = example.id
    x = example.comment_text.to(device)
    preds = model(x)
    subm_id = [*subm_id, *x_id.detach().cpu().numpy().ravel()]
    subm_preds = [*subm_preds, *sigmoid(preds.detach().cpu().numpy()[:,0].ravel())]
submission = pd.DataFrame.from_dict({
    'id': subm_id,
    'prediction': subm_preds
})
submission.to_csv('submission.csv', index=False)